In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
import seaborn as sns
import scanpy as sc
import matplotlib.pyplot as plt

In [2]:
raw = snakemake.input["STARsoloRaw"]
filtered = snakemake.input["STARsoloFiltered"]

# load data
dir_10x_raw = Path(raw).parent
adataRaw = sc.read_10x_mtx(dir_10x_raw, var_names="gene_symbols")
dir_10x_filtered = Path(filtered).parent
adataFilter = sc.read_10x_mtx(dir_10x_filtered, var_names="gene_symbols")
name = dir_10x_raw.parent.parent.stem + "_" + dir_10x_raw.parent.stem
df = pd.DataFrame()

# compute rankings
df["total_counts"] = adataRaw.to_df().sum(axis=1)
df["Filter"] = df.index.isin(adataFilter.to_df().index)
df = df.loc[df["total_counts"] > 0, :]
df = df.sort_values(by="total_counts", ascending=False)
df["rank"] = range(1, df.shape[0] + 1)

# plot
plt.clf()
fig = sns.lineplot(data=df, x="rank", y="total_counts", hue="Filter", palette="Blues")
fig.set(xscale="log", yscale="log", title=name, xlabel="Rank", ylabel="Total counts")
plt.show()

NameError: name 'snakemake' is not defined

In [ ]:
i = snakemake.input["STARsoloSummaries"]
# Load data, instantiate dataframe
df = pd.read_csv(i, header=None, names=["Metric", "Values"])
path = Path(i)
name = path.parent.parent.stem + "_" + path.parent.stem + "_" + path.name
df = df.set_index("Metric")

In [ ]:
# output table with sequencing metrics
sequencing = df[0:5].transpose()
sequencing["Number of Reads"] = sequencing["Number of Reads"].astype(int)
print(f"summarizing sequencing metrics...")
sequencing

In [ ]:
# output table with mapping metrics
mapping = df[5:11].drop(["Estimated Number of Cells"]).transpose()
if "Full" in name:
    mapping["Unique Reads in Cells Mapped to GeneFull"] = mapping[
        "Unique Reads in Cells Mapped to GeneFull"
    ].astype(int)
else:
    mapping["Unique Reads in Cells Mapped to Gene"] = mapping[
        "Unique Reads in Cells Mapped to Gene"
    ].astype(int)
print(f"summarizing mapping metrics...")
mapping

In [ ]:
# output table with cells metrics
cells = df[9:].transpose()
for column in cells:
    if column != "Fraction of Unique Reads in Cells":
        cells[column] = cells[column].astype(int)
print(f"summarizing cells metrics...")
cells